In [8]:
import json
import requests
import polars as pl

In [9]:
# Get latest data
response = requests.get(
    "https://ftp.ripe.net/pub/stats/ripencc/nro-stats/latest/nro-delegated-stats"
)
assert response.status_code == 200
rows = [x.split("|") for x in response.text.split("\n")][4:]
print(f"Retrieved {len(rows)} rows")

Retrieved 719443 rows


In [10]:
# Put in polars dataframe
registry_enum = pl.Enum(
    ["afrinic", "apnic", "arin", "lacnic", "ripe", "ripencc", "iana"]
)
type_enum = pl.Enum(["ipv4", "ipv6", "asn"])
status_enum = pl.Enum(["allocated", "assigned", "reserved", "available"])
lf = (
    pl.LazyFrame(
        rows,
        orient="row",
        schema={
            "registry": registry_enum,
            "cc": str,
            "type": type_enum,
            "ip": str,
            "count": pl.UInt32,
            "date": str,
            "status": status_enum,
            "summary": str,
            "extra": str,
        },
    )
    .filter(pl.col("type") == "ipv4")
    .drop(["summary", "extra"])
)

In [11]:
# Add some extra columns
lf = (
    lf.with_columns(
        pl.col("ip")
        .str.split_exact(by=".", n=3)
        .struct.rename_fields(["ip_byte_1", "ip_byte_2", "ip_byte_3", "ip_byte_4"])
        .alias("ip_split"),
    )
    .unnest("ip_split")
    .with_columns(
        (
            pl.col("ip_byte_1").cast(pl.UInt32) * (2**24)
            + pl.col("ip_byte_2").cast(pl.UInt32) * (2**16)
            + pl.col("ip_byte_3").cast(pl.UInt32) * (2**8)
            + pl.col("ip_byte_4").cast(pl.UInt32) * (2**0)
        ).alias("ip_integer")
    )
    .with_columns((pl.col("ip_integer").floordiv(256)).alias("ip_block_integer"))
    .with_columns(pl.col("count").floordiv(256).alias("block_count"))
    .sort("ip_integer")
    .drop(["ip_byte_1", "ip_byte_2", "ip_byte_3", "ip_byte_4"])
)

In [12]:
def group_consecutive_rows(
    lf: pl.DataFrame, column_name: str, first_value: str
) -> pl.DataFrame:
    return lf.with_columns(
        (pl.col(column_name) != pl.col(column_name).shift(1, fill_value=first_value))
        .cum_sum()
        .alias(f"{column_name}_group_id")
    )


lf = group_consecutive_rows(lf, "registry", "ripe")
lf = group_consecutive_rows(lf, "cc", "XY")
lf = group_consecutive_rows(lf, "status", "available")
lf.collect()

registry,cc,type,ip,count,date,status,ip_integer,ip_block_integer,block_count,registry_group_id,cc_group_id,status_group_id
enum,str,enum,str,u32,str,enum,u32,u32,u32,u32,u32,u32
"""iana""","""ZZ""","""ipv4""","""0.0.0.0""",16777216,"""19810901""","""reserved""",0,0,65536,1,1,1
"""apnic""","""AU""","""ipv4""","""1.0.0.0""",256,"""20110811""","""assigned""",16777216,65536,1,2,2,2
"""apnic""","""CN""","""ipv4""","""1.0.1.0""",256,"""20110414""","""assigned""",16777472,65537,1,2,3,2
"""apnic""","""CN""","""ipv4""","""1.0.2.0""",512,"""20110414""","""assigned""",16777728,65538,2,2,3,2
"""apnic""","""AU""","""ipv4""","""1.0.4.0""",1024,"""20110412""","""assigned""",16778240,65540,4,2,4,2
…,…,…,…,…,…,…,…,…,…,…,…,…
"""iana""","""ZZ""","""ipv4""","""251.0.0.0""",16777216,"""19810901""","""reserved""",4211081216,16449536,65536,11753,138641,11503
"""iana""","""ZZ""","""ipv4""","""252.0.0.0""",16777216,"""19810901""","""reserved""",4227858432,16515072,65536,11753,138641,11503
"""iana""","""ZZ""","""ipv4""","""253.0.0.0""",16777216,"""19810901""","""reserved""",4244635648,16580608,65536,11753,138641,11503
